In [227]:
import re
import pandas as pd

# Reading in data
in_file = open("data/district_data_23_Billion.csv", 'r')
county_file = open("data/EDGE_GEOCODE_PUBLICLEA_1920.csv", 'r')

school_dataframe = pd.read_csv(in_file)
county_dataframe = pd.read_csv(county_file, dtype={'CNTY':str})

# Renaming district code column to match
county_dataframe.rename(columns={'LEAID':'NCESID'}, inplace = True)

# Merging dataframes based onmatching district code
merged_dataframes = pd.merge(left = school_dataframe, right = county_dataframe, how = 'inner', on = 'NCESID')

# Taking columns needed from merged dataframe
data = merged_dataframes[['CNTY','NMCNTY','STATE','State and local revenue, per pupil, cost adjusted', 'Enrollment',
                          'Percent White', 'Student poverty rate', 'Median household income', 'Median property value']].copy()

# Removing unnecessary characters with regular expressions($,%)
data['State and local revenue, per pupil, cost adjusted'] = data['State and local revenue, per pupil, cost adjusted'].replace("[$,]", "", regex=True).astype(float)
data['Median household income'] = data['Median household income'].replace("[$,]", "", regex=True).astype(float)
data['Median property value'] = data['Median property value'].replace("[$,]", "", regex=True).astype(float)
data['Percent White'] = data['Percent White'].replace("[%]", "", regex=True).astype(int)
data['Enrollment'] = data['Enrollment'].replace("[,]", "", regex=True).astype(int)
data['Student poverty rate'] = data['Student poverty rate'].dropna().replace("[%]", "", regex=True).astype(int)

# Combining multiple districts into one county and calculating mean/sum of combined data
data.groupby('CNTY').agg({'State and local revenue, per pupil, cost adjusted':'mean', 'Percent White':'mean',
                          'Student poverty rate':'mean', 'Median household income':'mean', 'Median property value':'mean', 'Enrollment':'sum'})

# Fixing county codes
data.CNTY.str.zfill(5)

# Writing to output file
data.to_csv('data/county_data.csv', index=False)